In [1]:
import pandas as pd
from utils import get_beer_vector, get_beer_style_recommendations, splitDftoDict

In [53]:
beer_df = pd.read_gbq(project_id='scarlet-labs',
                      query="select * from `scarlet-labs.beer.beer_info_master_table` order by ratings desc",
                      dialect='standard')

Requesting query... ok.
Query running...
Query done.
Processed: 739.1 kb

Retrieving results...
Got 5517 rows.

Total time taken 2.61 s.
Finished at 2018-12-12 16:18:26.


In [43]:
import re
beer_style_clean = [re.sub(string=x, pattern="[(]|[)]", repl="").strip().replace(" / ", "_").replace(" ", "_").lower().strip() for x in sorted(beer_df['beer_style'].unique())]
beer_style_clean_df = pd.DataFrame({"beer_style":sorted(beer_df['beer_style'].unique()), "beer_style_clean":beer_style_clean}).set_index("beer_style")

In [48]:
#beer_df.set_index("beer_style").join(beer_style_clean_df).reset_index().to_gbq(project_id="scarlet-labs", destination_table="beer.beer_info_master_table2", if_exists="replace", verbose=False)

In [ ]:
#beer_df.to_gbq(project_id="scarlet-labs", destination_table="beer.beer_info_master_table", if_exists="replace", verbose=False)

In [54]:
beer_style_dict = splitDftoDict(df=beer_df, split_col="beer_style_clean")

In [ ]:
recs_df = pd.DataFrame()
for style in beer_df['beer_style_clean'].unique():
        print(style)
        recs = get_beer_style_recommendations(beer_style_dict[style])
        try:
            recs_df = pd.concat([recs_df, recs], axis=0, ignore_index=True)
        except:
            next
recs_df.to_gbq(project_id='scarlet-labs',
                        destination_table="beer.recommendations_raw",
                        if_exists="replace",
                        chunksize=200,
                        verbose=False)

In [60]:
recs_df.to_csv("test.csv", index=False)